
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>


<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.3 - alpha preview</p>
<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

Getting Underway - Downloading Datasets (Assets)
In this notebook, TODO: description

### Section 0: Import modules, and setup logging

In [2]:
# Standard imports
import logging
from pprint import pprint
import os
from pathlib import Path
import random
# Import mantaray and the Ocean API (squid)
import squid_py
from squid_py.ocean.ocean import Ocean
from squid_py.config import Config
import mantaray_utilities as manta_utils
from squid_py.keeper.web3_provider import Web3Provider
# Setup logging
manta_utils.logging.logger.setLevel('INFO')
from mantaray_utilities.user import password_map
# manta_utils.logging.logger.setLevel('DEBUG')
# os.environ['USE_K8S_CLUSTER'] = 'True' # Enable this for testing local -> AWS setup

In [3]:
# Get the configuration file path for this environment
CONFIG_INI_PATH = manta_utils.config.get_config_file_path()
logging.critical("Deployment type: {}".format(manta_utils.config.get_deployment_type()))
logging.critical("Configuration file selected: {}".format(CONFIG_INI_PATH))
logging.critical("Squid API version: {}".format(squid_py.__version__))

50 - <ipython-input-3-84136f362abd> - <module>        - Deployment type: DEFAULT
50 - <ipython-input-3-84136f362abd> - <module>        - Configuration file selected: /home/batman/ocn/mantaray_jupyter/config_local.ini
50 - <ipython-input-3-84136f362abd> - <module>        - Squid API version: 0.4.2


### Section 1: Instantiate a simulated User
A 'User' in an abstract class representing a user of Ocean Protocol


In [4]:
configuration = Config(CONFIG_INI_PATH)
ocn = Ocean(configuration)

20 - ocean_secret_store - __init__        - 	SecretStore: url http://localhost:12001, parity-client http://localhost:8545, account 0x068ed00cf0441e4829d9784fcbe7b9e26d4bd8d0
20 - diagnostics     - verify_contracts - Keeper contract artifacts (JSON abi files) at: artifacts_spree
20 - diagnostics     - verify_contracts - Using keeper contracts from network spree, network id is 8996
20 - diagnostics     - verify_contracts - Looking for keeper contracts ending with ".spree.json", e.g. "ServiceExecutionAgreement.spree.json".
20 - diagnostics     - verify_contracts - Finished loading keeper contracts:
	Dispenser: 0x8930D1F61Defb9CBD652136B4C3D93f19AC35678
	OceanToken: 0xe749E2f8482810b11B838Ae8c5eb69e54d610411
	DIDRegistry: 0x611f28Ef25D778aFC5a0034Aea94297e2c215a42
	ServiceExecutionAgreement: 0x312213d6f6b5FCF9F56B7B8946A6C727Bf4Bc21f
	PaymentConditions: 0xF9E633CBeEB2A474D3Fe22261046C99e805beeC4
	AccessConditions: 0xFe0145Caf0EC55D23dc1b08431b071f6e1123a76
20 - diagnostics     - check_depl

In [5]:
# Get a consumer account
path_passwords = manta_utils.config.get_project_path() / 'passwords.csv'
passwords = manta_utils.user.load_passwords(path_passwords)

consumer_acct = random.choice([acct for acct in ocn.accounts.list() if password_map(acct.address, passwords)])
consumer_acct.password = password_map(consumer_acct.address, passwords)
assert consumer_acct.password
print("Consumer account address: ", consumer_acct.address)

20 - user            - load_passwords  - 31 account-password pairs loaded


Consumer account address:  0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0


### Section 2: Find an asset

In [6]:
# Use the Query function to get all existing assets
basic_query = {"service":{"$elemMatch":{"metadata": {"$exists" : True }}}}
all_ddos = ocn.assets.query(basic_query)
assert len(all_ddos), "There are no assets registered, go to s03_publish_and_register!"
print("There are {} assets registered in the metadata store.".format(len(all_ddos)))

assert len(all_ddos), "There are no assets registered, go to s03_publish_and_register!"

# Get a DID for testing
selected_did = all_ddos[-1].did
print("Selected DID:",selected_did)

20 - ocean_assets    - query           - Searching asset query: {'service': {'$elemMatch': {'metadata': {'$exists': True}}}}


There are 27 assets registered in the metadata store.
Selected DID: did:op:abe7e86bedd34ef28e0436726039b67a4487bb5959fb4feba2b1c051e3d74d78


In [7]:
#TODO: The Asset class does not offer much beyond DDO class
#TODO: Term 'asset' is confusing here
this_asset = ocn.assets.resolve(selected_did)
pprint(this_asset)
print("Selected asset for access:", this_asset.metadata['base']['name'])
print("Price of asset:", this_asset.metadata['base']['price'])
print("Your balance:", ocn.accounts.balance(consumer_acct).ocn)

Selected asset for access: Ocean protocol white paper
Price of asset: 9
Your balance: 1091


Your account will need some Ocean Token to make real transactions

In [8]:
if ocn.accounts.balance(consumer_acct).ocn < 100:
    ocn.accounts.request_tokens(consumer_acct, 100)

Purchase the Asset!

In [9]:
#TODO: The service_definition_id will change to service_type
service_agreement_id = ocn.assets.order(this_asset.did, '0', consumer_acct)
print('New service agreement id:', service_agreement_id)

New service agreement id: 0x8530bf2a001e498b90c65db8139a6cb370fdc8e364904baea5a807e443087bc4


20 - utils           - process_tx_receipt - Success: got PaymentLocked event after fulfilling condition.
20 - brizo           - consume_service - invoke consume endpoint with this url: http://172.15.0.17:8030/api/v1/brizo/services/consume?url=https://testocnfiles.blob.core.windows.net/testfiles/testzkp.pdf&serviceAgreementId=0x8530bf2a001e498b90c65db8139a6cb370fdc8e364904baea5a807e443087bc4&consumerAddress=0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0
20 - brizo           - consume_service - Saved downloaded file in /home/batman/ocn/mantaray_jupyter/downloads_spree/datafile.abe7e86bedd34ef28e0436726039b67a4487bb5959fb4feba2b1c051e3d74d78.0/testzkp.pdf
20 - brizo           - consume_service - invoke consume endpoint with this url: http://172.15.0.17:8030/api/v1/brizo/services/consume?url=s3://ocean-test-osmosis-data-plugin-dataseeding-1537375953/data.txt&serviceAgreementId=0x8530bf2a001e498b90c65db8139a6cb370fdc8e364904baea5a807e443087bc4&consumerAddress=0x068Ed00cF0441e4829D9784fCBe7b9e26

The asset download is automatically initiated, this will take time to complete!

In [10]:
asset_path = Path.cwd() / ocn._config.downloads_path / f'datafile.{this_asset.asset_id}.0'
print("Check for your downloaded asset in", asset_path)
print("This might not appear immediately - the transaction needs be mined and the download needs to complete!")

Check for your downloaded asset in /home/batman/ocn/mantaray_jupyter/downloads_spree/datafile.abe7e86bedd34ef28e0436726039b67a4487bb5959fb4feba2b1c051e3d74d78.0
This might not appear immediately - the transaction needs be mined and the download needs to complete!
